In [2]:
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, StandardScaler

def transforming(columns, df):
    for column in columns:
        encoder = LabelEncoder()
        df[column] = encoder.fit_transform(df[column])


df = pd.read_csv("bird_migration_data.csv").drop(columns=["Bird_ID","Tagged_By", "Tag_Type", "Migration_Interrupted", "Interrupted_Reason", "Migration_Success", "Observation_Quality", "Tracking_Quality", "Recovery_Location_Known"], axis=1)


transforming(columns=["Species", "Region", "Habitat", "Weather_Condition", "Migration_Reason", "Nesting_Success","Migration_Start_Month", "Migration_End_Month", "Migrated_in_Flock", "Food_Supply_Level"], df=df)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers

model = keras.Sequential([
    keras.layers.Input(shape=(31,)),
    keras.layers.Dense(128, activation="relu", kernel_regularizer=regularizers.l2(0.01)),
    keras.layers.Dropout(0.3), 
    keras.layers.Dense(64, activation="relu", kernel_regularizer=regularizers.l2(0.01)),
    keras.layers.Dropout(0.3), 
    keras.layers.Dense(32, activation="relu", kernel_regularizer=regularizers.l2(0.01)),
    keras.layers.Dense(1)
])

In [5]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping # type: ignore


X = df.drop(["Flight_Distance_km"], axis=1)

scaler = StandardScaler()
numeric_cols = X.select_dtypes(include=['float64', 'int64']).columns
X[numeric_cols] = scaler.fit_transform(X[numeric_cols])

y = df["Flight_Distance_km"]
print(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

early_stopping = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True)
optimizer = keras.optimizers.Adam(learning_rate=0.005)
model.compile(optimizer = optimizer, loss = "mse", metrics=["mae"])
model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), batch_size=20, callbacks=[early_stopping])

0       1753.79
1       3263.05
2       2871.65
3       2317.38
4       3077.38
         ...   
9995    2357.69
9996    2901.75
9997    3019.80
9998    2663.78
9999    2549.41
Name: Flight_Distance_km, Length: 10000, dtype: float64
Epoch 1/100
400/400 ━━━━━━━━━━━━━━━━━━━━ 0s 514us/step - loss: 341343.4062 - mae: 463.9919 - val_loss: 295898.9062 - val_mae: 433.6316
Epoch 2/100
400/400 ━━━━━━━━━━━━━━━━━━━━ 0s 397us/step - loss: 340740.8750 - mae: 466.6474 - val_loss: 293578.4062 - val_mae: 432.2365
Epoch 3/100
400/400 ━━━━━━━━━━━━━━━━━━━━ 0s 395us/step - loss: 339842.8438 - mae: 465.0022 - val_loss: 288485.9062 - val_mae: 427.1080
Epoch 4/100
400/400 ━━━━━━━━━━━━━━━━━━━━ 0s 392us/step - loss: 332907.9688 - mae: 460.2422 - val_loss: 284219.3125 - val_mae: 424.7097
Epoch 5/100
400/400 ━━━━━━━━━━━━━━━━━━━━ 0s 402us/step - loss: 331833.9375 - mae: 460.3097 - val_loss: 285242.8125 - val_mae: 423.2511
Epoch 6/100
400/400 ━━━━━━━━━━━━━━━━━━━━ 0s 392us/step - loss: 335434.9688 - mae: 461.5873 - 